In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import keras.layers as layers
from keras.layers import Dense, Conv2D, Activation, MaxPooling2D, Flatten, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau
import math
import albumentations as A

# Model

stworzony na podstawie modelu z poprzednich części zadania.

In [2]:
f_mnist_model = tf.keras.Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(28, 28 ,1)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(350,activation='relu'),
    layers.Dense(150, activation='relu'),
    Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

f_mnist_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.4,
                              patience=3, min_lr=0.00000001)

# Przygotowanie danych.
Dane Fashion_Mnist są stworzone następująco: 60 000 przykładów treningowych oraz 10 000 przykładów testowych.
Na zestawie treningowym (60 000) przeprowadzę technikę augmentacji. Stworzę dodatkowe dane (w sumie zbiór danych treningowych będzie 5 razy liczniejszy), które następnie podzielę na zbiór treningowy i walidacyjny w zadanym stosunku 0.9 do 0.1. Na podstawie tych danych wytrenuję model.
Zestaw testowy pozostawiam do testowania wytrenowanego modelu.

## Augmentacja danych

In [3]:
def horizontal_flip (data):

    horizontal_flip_images = []
    for image in data:
      transform = A.HorizontalFlip(p=0.5)
      augmented_image = transform(image=image)['image']
      horizontal_flip_images.append(augmented_image)
    
    images = np.array(horizontal_flip_images)
    return images

def shift_scale_rotate (data):

    shift_scale_sotate_images = []
    for image in data:
      transform = A.ShiftScaleRotate(p=0.5)
      augmented_image = transform(image=image)['image']
      shift_scale_sotate_images.append(augmented_image)
    
    images = np.array(shift_scale_sotate_images)
    return images
  
def random_rotate (data):

    random_rotate_images = []
    for image in data:
      transform = A.RandomRotate90()
      augmented_image = transform(image=image)['image']
      random_rotate_images.append(augmented_image)
    
    images = np.array(random_rotate_images)
    return images 

def transpose (data):

    transpose_images = []
    for image in data:
      transform = A.Transpose()
      augmented_image = transform(image=image)['image']
      transpose_images.append(augmented_image)
    
    images = np.array(transpose_images)
    return images 

In [4]:
train, test = tf.keras.datasets.fashion_mnist.load_data()
images, labels = train
images = images/255.0
labels = labels.astype(np.int32)

4422102/4422102 [==============================] - 0s 0us/step


In [5]:
images_aug1 = horizontal_flip(images)
images_aug2 = shift_scale_rotate(images)
images_aug3 = random_rotate(images)
images_aug4 = transpose(images)

images_transformed = np.concatenate((images_aug1,images_aug2,images_aug3,images_aug4, images), axis=0)
labels_transformed = np.concatenate((labels, labels, labels, labels, labels), axis=None)

In [6]:
print(images_transformed.shape)
print(labels_transformed.shape)

(300000, 28, 28)
(300000,)


In [7]:
X_train, X_val, y_train, y_val = \
    train_test_split(images_transformed, labels_transformed, test_size=0.1, random_state=4321, stratify=labels_transformed)

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.batch(150)

train_stats = f_mnist_model.fit(train_ds, epochs=30,callbacks=[reduce_lr],validation_data=(X_val, y_val))

Epoch 1/30
1800/1800 [==============================] - 19s 6ms/step - loss: 0.5983 - accuracy: 0.7857 - val_loss: 0.3865 - val_accuracy: 0.8546 - lr: 0.0010
Epoch 2/30
1800/1800 [==============================] - 11s 6ms/step - loss: 0.3601 - accuracy: 0.8708 - val_loss: 0.3146 - val_accuracy: 0.8830 - lr: 0.0010
Epoch 3/30
1800/1800 [==============================] - 11s 6ms/step - loss: 0.2918 - accuracy: 0.8943 - val_loss: 0.2741 - val_accuracy: 0.9006 - lr: 0.0010
Epoch 4/30
1800/1800 [==============================] - 11s 6ms/step - loss: 0.2491 - accuracy: 0.9089 - val_loss: 0.2637 - val_accuracy: 0.9032 - lr: 0.0010
Epoch 5/30
1800/1800 [==============================] - 11s 6ms/step - loss: 0.2180 - accuracy: 0.9203 - val_loss: 0.2382 - val_accuracy: 0.9137 - lr: 0.0010
Epoch 6/30
1800/1800 [==============================] - 11s 6ms/step - loss: 0.1932 - accuracy: 0.9291 - val_loss: 0.2481 - val_accuracy: 0.9130 - lr: 0.0010
Epoch 7/30
1800/1800 [==============================

# Dokładność modelu
Na podstawie danych testowych pochodzących z pierwotnego (bez augmentacji) zbioru danych Fashion_Mnist

In [8]:
train, test = tf.keras.datasets.fashion_mnist.load_data()
images, labels = test
images = images/255.0
labels = labels.astype(np.int32)
score = f_mnist_model.evaluate(images, labels)
score

313/313 [==============================] - 1s 3ms/step - loss: 0.7868 - accuracy: 0.9211


[0.7868013978004456, 0.9211000204086304]

# Wnioski:

Uważam, że nie jest źle. Model bywał przetrenowywany, więc w celu złagodzenia tej sytuacji zastosowałam w modelu Dropout().

W internecie (przynajmniej tam, gdzie ja dotarłam) rzadko kto zwraca uwagę na accuracy na danych testowych. Wystarcza, że sam wytrenowany na danych treningowych model ma wysoką dokładność i nie zwraca się uwagi na przetrenowanie modelu. W tych okolicznościach osiągnięcie przez laika (czyli mnie) accuracy ~.92 na danych testowych uważam za sukces.

Zauważyłam też, że powszechnie błędnie stosuje się technikę augmentacji. Zazwyczaj ludzie zmieniali dane i jako takie zmienione trenowali. Nie dodawali wykreowanych danych do podstawowego datasetu tylko trenowali modele na przekształconych danych. Nie zwiększali liczebności danych treningowych, a to na tym, zdaje się, polega augmentacja.